# Age and Gender Classification


Multi class classification problem.

http://www.openu.ac.il/home/hassner/projects/cnn_agegender/CNN_AgeGenderEstimation.pdf

**Y variable**

- MALE
- FEMALE
- 0-2
- 4-6
- 8-13
- 15-20
- 25-32
- 38-43
- 48-53
- 60-

10 Class, 2 for Gender and 8 for Age

Keras does not implement Local Response Normalizing, we have substituted Batch Normalization here.



## Load the images from HDF5

In [6]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import random

hdf5_path = '../../data/Adience/hdf5/adience.h5'
hdf5_file = h5py.File(hdf5_path, mode = 'r')


data_size = hdf5_file['train_images'].shape[0]
batch_size = 256
nb_class = 10

print("Total Training Images {}".format(data_size))

Total Training Images 16228


In [7]:
x_train = hdf5_file['train_images']
y_train = hdf5_file['train_labels']


# One shot encoding of Y

In [8]:
from keras.utils import np_utils

y_train_1 = y_train[:,0]
y_train_2 = y_train[:,1]



y_train_1_onecoding = np_utils.to_categorical(y_train_1, 8)
y_train_2_onecoding = np_utils.to_categorical(y_train_2, 2)

y_train_onecoding = np.concatenate((y_train_2_onecoding, y_train_1_onecoding), axis = 1)

y_train_onecoding

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [9]:
x_train.shape

(16228, 256, 256, 3)

In [10]:
from keras.layers import Input, Conv2D, Dense,MaxPooling2D, Flatten, Activation,Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.backend import tf as ktf
import numpy as np
np.random.seed(123)




def age_gender_model():
    

    input_shape = (256, 256, 3)
    
    x_input = Input(input_shape)
    
    # Conv Layer 1
    
    x = Conv2D(filters = 12, kernel_size = (7,7), strides = (1,1), \
               padding = "valid", kernel_initializer='glorot_uniform')(x_input)
    
    x = Activation("relu")(x)
    
    x = MaxPooling2D(pool_size = (3,3), strides = (1,1))(x)
    
    #x = BatchNormalization()(x)
    
    # Conv Layer 2
    
    x = Conv2D(filters = 24, kernel_size = (5,5), strides = (1,1), 
               padding = "valid",kernel_initializer='glorot_uniform')(x)
    
    x = Activation("relu")(x)


    x = MaxPooling2D(pool_size = (3,3), strides = (1,1))(x)
    
    #x = BatchNormalization()(x)

    # Conv Layer 3
    
    x = Conv2D(filters = 36, kernel_size = (3,3), strides = (1,1), 
               padding = "valid",kernel_initializer='glorot_uniform')(x)
    
    x = Activation("relu")(x)


    x = MaxPooling2D(pool_size = (3,3), strides = (1,1))(x)
    
    #x = BatchNormalization()(x)
    
    
    x = Flatten()(x)
    
    x = Dense(512, activation = "relu")(x)
    
    x = Dropout(rate = 0.5)(x)
    
    x = Dense(512, activation ="relu")(x)
    
    x = Dropout(rate = 0.5)(x)

    
    predictions = Dense(10, activation="softmax")(x)
    
    model = Model(inputs = x_input, outputs = predictions)

    
    return model

    


    




In [11]:
from keras import optimizers
from keras.callbacks import History, EarlyStopping



# Optimizer
sgd = optimizers.SGD(lr=0.005, momentum=0, decay=1e-6, nesterov=False)


# Callbacks
early_stop_th = 10**-7
callbacks = [EarlyStopping(monitor='acc', min_delta=early_stop_th, patience=5, verbose=0, mode='auto')]



model = age_gender_model()
batch_size = 64
epochs = 100

model.compile(optimizer = "sgd", loss = "categorical_crossentropy", metrics = ["accuracy"])


x_train_small = x_train[1:500]
y_train_small = y_train_onecoding[1:500]

hist = model.fit(x_train_small, y_train_small, batch_size=32,validation_split = 0.1,
                     epochs=epochs, callbacks = callbacks, verbose = 1)


# fits the model on batches with real-time data augmentation:
#hist = model.fit_generator(datagen.flow(x_train, y_train_onecoding, batch_size=32),validation_split = 0.1,
#                    steps_per_epoch=len(x_train) / 32, epochs=epochs, callbacks = callbacks, verbose = 1)



Train on 449 samples, validate on 50 samples
Epoch 1/100
192/449 [===========>..................] - ETA: 17:47 - loss: 18.7831 - acc: 0.0052

KeyboardInterrupt: 

## Pre-processing of the image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)




In [ ]:
def plot_training(hist):

    fig = plt.figure(figsize=(8,6))
    plt.tight_layout()
    plt.subplot(121)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title("Train Loss and Validation Loss")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.legend(loc='upper left')
    plt.subplot(122)
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title("Train Accuracy and Validation Accurcy")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.legend(loc='upper left')



plot_training(hist)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))